# Chess Engine with TensorFlow

## Dataset

In [1]:
import os

files = [file for file in os.listdir("../../data/pgn") if file.endswith(".pgn")]

In [2]:
len(files)

79

In [3]:
from chess import pgn

def load_pgn(file_path):
    games = []
    with open(file_path, 'r') as pgn_file:
        while True:
            game = pgn.read_game(pgn_file)
            if game is None:
                break
            games.append(game)
    return games

In [4]:
from tqdm import tqdm

LIMIT_OF_FILES = min(len(files), 24)
games = []
i = 1
for file in tqdm(files):
    games.extend(load_pgn(f"../../data/pgn/{file}"))
    if (i >= LIMIT_OF_FILES):
        break
    i += 1

 29%|██▉       | 23/79 [00:20<00:50,  1.12it/s]


In [5]:
len(games)

10327

## Build & train a neural network

In [6]:
import numpy as np
from chess import Board
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam # type: ignore
import time

In [7]:
def board_to_matrix(board: Board):
    matrix = np.zeros((8, 8, 12))
    piece_map = board.piece_map()
    for square, piece in piece_map.items():
        row, col = divmod(square, 8)
        piece_type = piece.piece_type - 1
        piece_color = 0 if piece.color else 6
        matrix[row, col, piece_type + piece_color] = 1
    return matrix


def create_input_for_nn(games):
    X = []
    y = []
    for game in games:
        board = game.board()
        for move in game.mainline_moves():
            X.append(board_to_matrix(board))
            y.append(move.uci())
            board.push(move)
    return X, y


def encode_moves(moves):
    move_to_int = {move: idx for idx, move in enumerate(set(moves))}
    return [move_to_int[move] for move in moves], move_to_int

In [9]:
X, y = create_input_for_nn(games)
y, move_to_int = encode_moves(y)
y = to_categorical(y, num_classes=len(move_to_int))
X = np.array(X)

In [10]:
model = Sequential([
    Conv2D(64, (3, 3), activation='relu', input_shape=(8, 8, 12)),
    Conv2D(128, (3, 3), activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dense(len(move_to_int), activation='softmax')
])

c:\Users\Siddharth\anaconda3\envs\kaggle_chess\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [11]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 6, 6, 64)       │         6,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 4, 4, 128)      │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1867)           │       479,819 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,085,195 (4.14 MB)

 Trainable params: 1,085,195 (4.14 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(X, y, epochs=100, validation_split=0.1, batch_size=64)
model.save("models/TF_100EPOCHS.keras")

Epoch 1/100
11931/11931 ━━━━━━━━━━━━━━━━━━━━ 218s 18ms/step - accuracy: 0.0892 - loss: 5.6085 - val_accuracy: 0.1530 - val_loss: 4.3998
Epoch 2/100
11931/11931 ━━━━━━━━━━━━━━━━━━━━ 172s 14ms/step - accuracy: 0.1698 - loss: 4.1692 - val_accuracy: 0.1733 - val_loss: 4.1208
Epoch 3/100
11931/11931 ━━━━━━━━━━━━━━━━━━━━ 184s 13ms/step - accuracy: 0.1910 - loss: 3.8622 - val_accuracy: 0.1752 - val_loss: 4.0250
Epoch 4/100
11931/11931 ━━━━━━━━━━━━━━━━━━━━ 79s 7ms/step - accuracy: 0.2035 - loss: 3.7048 - val_accuracy: 0.1804 - val_loss: 3.9896
Epoch 5/100
11931/11931 ━━━━━━━━━━━━━━━━━━━━ 96s 8ms/step - accuracy: 0.2141 - loss: 3.6076 - val_accuracy: 0.1824 - val_loss: 3.9719
Epoch 6/100
11931/11931 ━━━━━━━━━━━━━━━━━━━━ 75s 6ms/step - accuracy: 0.2198 - loss: 3.5385 - val_accuracy: 0.1832 - val_loss: 3.9686
Epoch 7/100
11931/11931 ━━━━━━━━━━━━━━━━━━━━ 76s 6ms/step - accuracy: 0.2254 - loss: 3.4859 - val_accuracy: 0.1876 - val_loss: 3.9600
Epoch 8/100
11931/11931 ━━━━━━━━━━━━━━━━━━━━ 72s 6ms/ste

## Predictions

In [ ]:
# Load the trained model
from tensorflow.keras.models import load_model
model = load_model("../../models/TF_50EPOCHS.keras")

In [ ]:
int_to_move = dict(zip(move_to_int.values(), move_to_int.keys()))


def predict_next_move(board):
    board_matrix = board_to_matrix(board).reshape(1, 8, 8, 12)
    predictions = model.predict(board_matrix)[0]
    legal_moves = list(board.legal_moves)
    legal_moves_uci = [move.uci() for move in legal_moves]
    sorted_indices = np.argsort(predictions)[::-1]
    for move_index in sorted_indices:
        move = int_to_move[move_index]
        if move in legal_moves_uci:
            return move
    return None

In [ ]:
# Create a chess board (start position)
board = Board()

In [ ]:
# Display the board before prediction
print("Board before prediction:")
print(board)

# Predict and make the move
next_move = predict_next_move(board)
board.push_uci(next_move)

# Display the board after prediction
print("\nPredicted move:", next_move)
print("Board after prediction:")
print(board)

In [ ]:
print(str(pgn.Game.from_board(board)))